In [1]:
#Change cwd:
from pathlib import Path
import os

os.chdir("..")
print(os.getcwd())

C:\Users\lukev\Documents\Thesis\EBM-NLP\pio_ebm_nlp


### Load data and model

In [2]:
import spacy
import numpy as np
from spacy import displacy
from spacy.scorer import Scorer

print(spacy.__version__) #Should be >= 3.1.3 and < 3.2.0

def get_entities(annotations): #Generate entities from training data
    annot_list = list()
    for i in range(len(annotations)):
        annot = []
        for j in range(len(annotations[i])):
            label = annotations[i][j][0]
            if(label != "None"): 
                annot.append(annotations[i][j])
        annot_list.append(annot)
    return annot_list
    

nlp1 = spacy.load(R".\spacy_acc\output\model-best") #load the model
texts_train = np.load("data/full_texts.npy", allow_pickle = True)
ann_train = np.load("data/data_standard.npy", allow_pickle = True)
file_names = np.load("data/file_names_dev.npy", allow_pickle = True)
stand_off_train = np.load("data/stand_off_ann.npy", allow_pickle = True)
indices = np.load("data/indices.npy", allow_pickle = True)
val_size = int(0.10 * len(texts_train)) #Size of dev set
tokens = [[x[1] for x in X] for X in ann_train]
tokens_val = tokens[:val_size]
ann_train = get_entities(ann_train)
file_names_val = file_names[:val_size]
texts_val, ann_val = texts_train[:val_size], ann_train[:val_size]


3.1.3


### Visualize annotations for specified index (appendix A)

In [3]:
index = 30 #Index of abstract in appendix A
doc = nlp1(str(texts_val[index][1]))
entities = ["Population", "Intervention", "Outcome"]
entitieshier = ['P: Age', 'P: Sex', 'P: Sample size', 
                 'P: Condition', 'I: Surgical', 'I: Physical', 
                 'I: Drug', 'I: Educational', 'I: Psychological', 
                 'I: Other', 'I: Control', 'O: Physical', 'O: Pain', 
                 'O: Mortality', 'O: Adverse effects', 'O: Mental', 'O: Other']
colors = {'Population': '#2FAE9F', 'Intervention': '#2F9C53', 'Outcome': '#E8D35B'}
colorshier = {'P: Age': '#2FAEFF', 'P: Sex': '#2FAECF', 'P: Sample size': '#2FAEDF', 
                 'P: Condition': '#2FAEEF', 'I: Surgical': '#FFD23C', 'I: Physical': '#FCC73C', 
                 'I: Drug': '#F7C23C', 'I: Educational': '#F5BD1F', 'I: Psychological': '#F1A73C', 
                 'I: Other': '#EDA43C', 'I: Control': '#E6A13C', 'O: Physical': '#32BA32', 
                 'O: Pain': '#32BB32', 'O: Mortality': '#32CC32', 'O: Adverse effects': '#32DD32',
                 'O: Mental': '#32EE32', 'O: Other': '#32FF32'}
options = {"ents": entities, "colors": colors }
optionshier = {"ents": entitieshier, "colors": colorshier }
ents = []
for i, off_sets in enumerate(stand_off_train[index]):
    span = doc.char_span(off_sets[0], off_sets[1], off_sets[2])
    ents.append(span)
doc.set_ents(ents)
displacy.render(doc, style="ent", jupyter = True, options = optionshier)

### Sample span

In [4]:
doc = nlp1("We have compared the efficacy of aspirin in comparison to a placebo for the prevention of headaches in children aged 8-16 years.")
displacy.render(doc, style="ent", jupyter = True, options = optionshier)

### Generate sample span for different annotation strategies

In [5]:
from spacy.tokens import Span
doc1 = nlp1("Intravenous Immunoglobulin Therapy (IVIg) compared to a placebo injection for male children aged 8-12 years")
doc2 = nlp1("Intravenous Immunoglobulin Therapy (IVIg) compared to a placebo injection for male children aged 8-12 years")
doc3 = nlp1("Intravenous Immunoglobulin Therapy (IVIg) compared to a placebo injection for male children aged 8-12 years")
doc1.set_ents([Span(doc1, 1, 2, "I: Drug"), Span(doc1, 2, 3, "I: Drug"), Span(doc1, 3, 4, "I: Drug"), 
               Span(doc1, 4, 5, "I: Drug"), Span(doc1, 5, 6, "I: Drug"), Span(doc1, 9, 10, "I: Control"), 
              Span(doc1, 12, 13, "P: Sex"), Span(doc1, 13, 14, "P: Age"),Span(doc1, 15, 18, "P: Age"), 
               Span(doc1, 18, 19, "P: Age") ])
doc2.set_ents([Span(doc2, 1, 2, "I: Drug"), Span(doc2, 9, 10, "I: Control"), 
              Span(doc2, 12, 13, "P: Sex"), Span(doc2, 15, 18, "P: Age")])
doc3.set_ents([Span(doc3, 0, 1, "I: Drug"), Span(doc3, 1, 2, "I: Drug"), Span(doc3, 2, 3, "I: Drug"), 
               Span(doc3, 3, 4, "I: Drug"), Span(doc3, 4, 5, "I: Drug"), Span(doc3, 5, 6, "I: Drug"), 
               Span(doc3, 9, 10, "I: Control"), Span(doc3, 10, 11, "I: Control"), Span(doc3, 12, 13, "P: Sex"),
               Span(doc3, 13, 14, "P: Age"), Span(doc3, 14, 15, "P: Age"), Span(doc3, 15, 18, "P: Age"), 
               Span(doc3, 18, 19, "P: Age") ])
print("\n")
displacy.render(doc1, style="ent", jupyter = True, options = optionshier)
print("\n")
displacy.render(doc2, style="ent", jupyter = True, options = optionshier)
print("\n")
displacy.render(doc3, style="ent", jupyter = True, options = optionshier)
print("\n")

## Round 1 Sampling


In [6]:
import numpy as np

#Generate the ratio of fn for each abstract
def get_ratio_of_FN(entities_model, entities_groundtruth, tokens):
    fn_list = []
    text_lengths = [len(i) for i in tokens]
    for i in range(len(entities_model)): 
        if((text_lengths[i]) > 100): #Filter out abstracts with fewer than a 100 words
            fn = 0 #Number of fn in each abstract
            groundtruth_ents = entities_groundtruth[i]
            for entity in groundtruth_ents:
                if(not entity in entities_model[i]):
                    fn += 1
            fn_list.append((i, fn, (fn/text_lengths[i]), text_lengths[i]))
    return fn_list

entities_model = []
entities_groundtruth = []
for ann in ann_val:
    entities_groundtruth.append(ann)
for texts in texts_val:
    doc3 = nlp1(str(texts))
    ents = list(doc3.ents)
    ents = [(ent.label_,ent.text) for ent in ents]
    entities_model.append(ents)
#Get fn scores
fn_list = get_ratio_of_FN(entities_model, entities_groundtruth, tokens_val)
#Sort on fn ratio
fn_list.sort(key=lambda x:x[2], reverse = True) #Sort on ratio of FN
print(fn_list[:10]) #Print first 10 sampled abstracts

        
    

[(244, 114, 0.397212543554007, 287), (16, 81, 0.39705882352941174, 204), (144, 48, 0.3221476510067114, 149), (320, 41, 0.3014705882352941, 136), (363, 95, 0.286144578313253, 332), (300, 49, 0.28488372093023256, 172), (268, 86, 0.27388535031847133, 314), (403, 39, 0.2708333333333333, 144), (129, 77, 0.2664359861591695, 289), (27, 67, 0.2637795275590551, 254)]


In [7]:
def get_first_x_documents(fn_list, file_names_val, amount):
    #Generates filenames
    files = []
    for i in range(amount):
        files.append(file_names_val[fn_list[i][0]])
    return files
def display_first_x_documents(fn_list, amount):
    for i in range(amount):
        index_text = fn_list[i][0]
        text = texts_val[index_text][1]
        doc = nlp1(str(text))
        displacy.render(doc, style="ent", jupyter = True, options = optionshier)
        
#Display the sampled abstracts with annotations      
display_first_x_documents(fn_list, 10)




In [8]:
indices = [i[0] for i in fn_list[:10]]
print(indices) #Indices of abstracts in dev set
print(texts_val[indices]) #Full-texts of sampled abstracts

[244, 16, 144, 320, 363, 300, 268, 403, 129, 27]
[['11210885.txt'
  'Alternative voice after laryngectomy using a sound-producing voice prosthesis.\n\nOBJECTIVE To improve the voice quality of female laryngectomees and/or laryngectomees with a hypotonic pharyngoesophageal (PE) segment by means of a pneumatic artificial source of voice incorporated in a regular tracheoesophageal (TE) shunt valve.\nSTUDY DESIGN Experimental, randomized, crossover trial.\nMETHODS The new sound source consists of a single silicone lip, which performs an oscillatory movement driven by expired pulmonary air flowing along the outward-striking lip through the TE shunt valve. A prototype of this pneumatic sound source is evaluated in vitro and in six laryngectomees. In vivo evaluation includes speech rate, maximal phonation time, perceptual voice evaluation of read-aloud prose by an expert listener, speech intelligibility measurements with 12 listeners, and self-assessment by the patients. Moreover, extensive a

## Round 2 Sampling


In [9]:
from collections import Counter

#Generate list of main interventions for each abstract
def get_main_interventions(ann_val):
    main_interventions = []
    for i, annotations in enumerate(ann_val):
        entities = [i[0] for i in annotations if i[0].startswith("I:")] #Only consider interventions
        entity_count = Counter(entities)
        try:
            most_common = entity_count.most_common(1)[0][0] #Select most common interventions
            main_interventions.append([most_common, i])
        except: #No interventions annotated in abstract
            main_interventions.append(["None", i]) 
            print(f"No interventions in annotations for abstract {i}, setting main intervention as 'None'...") 
    return main_interventions

main_interventions = get_main_interventions(ann_val)
text_lengths = [len(i) for i in tokens_val]

#Previously sampled abstracts
indices_to_remove = indices

#Also remove abstract from list if < 100 words or > 500 words.
def remove_outliers(abstracts, text_lengths):
    for i,length in enumerate(text_lengths):
        if (length < 100) or (length > 400):
            indices_to_remove.append(i)

remove_outliers(main_interventions, text_lengths)
print(f"\nIndices that will be rmeoved: {indices_to_remove}")
for index in sorted(indices_to_remove, reverse=True):
    del main_interventions[index]
    
#Main intervention counts for dev set
interventions = [i[0] for i in main_interventions]
print(f"\nMain intervention counts: {Counter(interventions)}")


No interventions in annotations for abstract 103, setting main intervention as 'None'...
No interventions in annotations for abstract 161, setting main intervention as 'None'...
No interventions in annotations for abstract 175, setting main intervention as 'None'...
No interventions in annotations for abstract 277, setting main intervention as 'None'...

Indices that will be rmeoved: [244, 16, 144, 320, 363, 300, 268, 403, 129, 27, 1, 17, 21, 31, 32, 34, 40, 45, 49, 74, 75, 81, 82, 84, 87, 91, 94, 101, 108, 110, 112, 118, 121, 123, 135, 138, 154, 161, 164, 178, 181, 182, 185, 192, 193, 211, 212, 213, 217, 227, 228, 238, 242, 267, 269, 272, 277, 281, 286, 289, 296, 304, 314, 318, 321, 325, 332, 333, 343, 350, 355, 359, 367, 368, 371, 374, 384, 386, 388, 390, 406, 408, 426, 438, 441]

Main intervention counts: Counter({'I: Drug': 218, 'I: Physical': 56, 'I: Educational': 43, 'I: Surgical': 21, 'I: Other': 11, 'I: Psychological': 6, 'I: Control': 3, 'None': 2})


### Taking sample for round 2

In [10]:
import random 
random.seed(5) #Set random seed
def random_sample(amount, abstracts, subject):
    relevant_abstracts = [i for i in abstracts if i[0] == subject] #Filter on subject
    sample = random.sample(relevant_abstracts, amount) #Tale the sample for subject
    return sample

#List of possible subjects:
list_of_subjects = ["I: Drug", "I: Physical",  "I: Educational", "I: Surgical" ,  "I: Other", "I: Psychological"]  
full_sample = []
amount = 2 #Amount of sampled abstracts
for subject in list_of_subjects:
    sample = random_sample(amount, main_interventions, subject)
    full_sample += sample
    
#Sampled abstracts for each subject:
print(f"Index of the sampled abstracts for each subject: {full_sample}")

Index of the sampled abstracts for each subject: [['I: Drug', 327], ['I: Drug', 125], ['I: Physical', 365], ['I: Physical', 139], ['I: Educational', 434], ['I: Educational', 393], ['I: Surgical', 33], ['I: Surgical', 280], ['I: Other', 157], ['I: Other', 14], ['I: Psychological', 134], ['I: Psychological', 25]]


In [11]:
indices = [i[1] for i in full_sample]
print(indices)
def display_documents(indices, texts_val):
    for index in indices:
        text = texts_val[index][1]
        print(len(text))
        print(ann_val[index])
        doc = nlp1(str(text))
        displacy.render(doc, style="ent", jupyter = True, options = optionshier)
        print("\n\n")
        
#Display sampled abstracts
display_documents(indices, texts_val)

[327, 125, 365, 139, 434, 393, 33, 280, 157, 14, 134, 25]
1112
[('I: Drug', 'PNU-142633'), ('I: Drug', ','), ('I: Drug', 'a'), ('I: Drug', 'selective'), ('I: Drug', '5-HT1D'), ('I: Drug', 'agonist'), ('P: Condition', 'acute'), ('P: Condition', 'migraine'), ('I: Control', 'matching'), ('I: Control', 'placebo'), ('P: Condition', 'acute'), ('P: Condition', 'migraine'), ('I: Drug', 'PNU-142633'), ('O: Physical', 'headache'), ('O: Physical', 'relief'), ('O: Pain', 'free'), ('O: Pain', 'of'), ('O: Pain', 'headache'), ('O: Pain', 'pain'), ('O: Pain', '.'), ('O: Adverse effects', 'Adverse'), ('O: Adverse effects', 'events'), ('O: Adverse effects', 'chest'), ('O: Adverse effects', 'pain'), ('O: Physical', 'QTc'), ('O: Physical', 'prolongation'), ('I: Drug', '5-HT'), ('I: Drug', 'agonist')]





1494
[('O: Other', 'Efficacy'), ('O: Other', 'safety'), ('I: Drug', 'perindopril/indapamide'), ('I: Drug', 'combination'), ('P: Condition', 'essential'), ('P: Condition', 'hypertension'), ('O: Other', 'efficacy'), ('O: Other', 'safety'), ('I: Drug', 'low'), ('I: Drug', 'dose'), ('I: Drug', 'combination'), ('I: Drug', 'perindopril'), ('I: Drug', '2'), ('I: Drug', 'mg/indapamide'), ('I: Drug', 'atenolol'), ('P: Sample size', '446'), ('P: Condition', 'hypertensive'), ('P: Age', 'mean'), ('P: Age', 'age'), ('P: Age', ':'), ('P: Age', '55.8'), ('P: Age', '+/-'), ('P: Age', '11.0'), ('P: Age', 'years'), ('I: Drug', 'Per/Ind'), ('I: Drug', 'Ate'), ('O: Physical', 'changes'), ('O: Physical', 'in'), ('O: Physical', 'trough'), ('O: Physical', 'supine'), ('O: Physical', 'systolic'), ('O: Physical', 'and'), ('O: Physical', 'diastolic'), ('O: Physical', 'blood'), ('O: Physical', 'pressure'), ('O: Physical', '('), ('O: Physical', 'sSBP'), ('O: Physical', ','), ('O: Physical', 'sDBP'), ('O: Physic




1728
[('I: Physical', 'Low'), ('I: Physical', 'Energy'), ('I: Physical', 'Safety'), ('I: Physical', 'Study'), ('I: Physical', 'implantable'), ('I: Physical', 'cardioverter'), ('I: Physical', 'defibrillators'), ('I: Surgical', '('), ('I: Surgical', 'ICDs'), ('I: Surgical', ')'), ('I: Physical', 'Low'), ('I: Physical', 'Energy'), ('I: Physical', 'Safety'), ('I: Physical', 'Study'), ('P: Condition', 'Patients'), ('P: Condition', 'with'), ('P: Condition', 'standard'), ('P: Condition', 'ICD'), ('P: Condition', 'indications'), ('I: Physical', 'defibrillation'), ('I: Physical', 'threshold'), ('I: Physical', 'testing'), ('I: Physical', '('), ('I: Physical', 'DFT'), ('I: Physical', ')'), ('I: Physical', 'ICD'), ('I: Physical', 'implant'), ('I: Physical', 'initial'), ('I: Physical', '2'), ('I: Physical', 'shocks'), ('I: Physical', 'for'), ('I: Physical', 'ventricular'), ('I: Physical', 'fibrillation'), ('I: Physical', 'conversion'), ('I: Physical', 'programmed'), ('I: Physical', 'at'), ('I: P




2233
[('I: Physical', 'recombinant'), ('I: Physical', 'human'), ('I: Physical', 'granulocyte'), ('I: Physical', 'colony-stimulating'), ('I: Physical', 'factor'), ('I: Physical', 'chemotherapy'), ('P: Condition', 'childhood'), ('P: Condition', 'acute'), ('P: Condition', 'lymphoblastic'), ('P: Condition', 'leukemia'), ('I: Physical', 'recombinant'), ('I: Physical', 'human'), ('I: Physical', 'granulocyte'), ('I: Physical', 'colony-stimulating'), ('I: Physical', 'factor'), ('I: Physical', '('), ('I: Physical', '['), ('I: Physical', 'G-CSF'), ('I: Physical', ']'), ('I: Physical', 'lenogastrim'), ('I: Physical', ')'), ('I: Physical', 'consolidation'), ('I: Physical', 'chemotherapy'), ('P: Age', 'childhood'), ('P: Condition', 'acute'), ('P: Condition', 'lymphoblastic'), ('P: Condition', 'leukemia'), ('P: Condition', 'ALL'), ('P: Sample size', 'Sixty-seven'), ('P: Age', 'children'), ('I: Physical', 'chemotherapy'), ('I: Drug', 'high-dose'), ('I: Drug', 'cytarabine'), ('I: Drug', 'etoposide'




1690
[('O: Pain', 'Pain'), ('I: Physical', 'chemotherapy'), ('O: Pain', 'pain'), ('I: Educational', 'conventional'), ('I: Educational', 'care'), ('I: Educational', 'alone'), ('I: Educational', 'conventional'), ('I: Educational', 'care'), ('I: Educational', 'plus'), ('I: Educational', 'a'), ('I: Educational', 'nursing'), ('I: Educational', 'intervention'), ('P: Condition', 'cancer'), ('I: Psychological', 'nursing'), ('I: Psychological', 'intervention'), ('I: Psychological', 'utilizing'), ('I: Psychological', 'problem-solving'), ('I: Psychological', 'approaches'), ('I: Psychological', 'to'), ('I: Psychological', 'symptom'), ('I: Psychological', 'management'), ('I: Psychological', 'and'), ('I: Psychological', 'improving'), ('O: Physical', 'physical'), ('O: Physical', 'functioning'), ('I: Psychological', 'and'), ('O: Mental', 'emotional'), ('O: Mental', 'health'), ('P: Sample size', '53'), ('P: Sample size', '60'), ('O: Pain', 'pain'), ('O: Physical', 'fatigue'), ('O: Pain', 'Pain'), ('

...




1582
[('I: Surgical', 'transurethral'), ('I: Surgical', 'resection'), ('I: Surgical', 'transurethral'), ('I: Surgical', 'resection'), ('I: Surgical', 'of'), ('I: Surgical', 'the'), ('I: Surgical', 'prostate'), ('I: Surgical', '('), ('I: Surgical', 'TURP'), ('I: Surgical', ')'), ('P: Sample size', 'Fifty-six'), ('P: Sex', 'male'), ('P: Age', '71.2'), ('P: Age', '+/-'), ('P: Age', '8.2'), ('P: Condition', 'TURP'), ('P: Sample size', '27'), ('I: Physical', 'room'), ('I: Physical', 'temperature'), ('I: Physical', 'irrigation'), ('I: Physical', 'fluid'), ('I: Physical', '('), ('I: Physical', '70'), ('I: Physical', 'degrees'), ('I: Physical', 'F'), ('I: Physical', ')'), ('I: Physical', 'throughout'), ('I: Surgical', 'TURP'), ('P: Sample size', '29'), ('I: Drug', 'warmed'), ('I: Drug', 'irrigation'), ('I: Drug', 'fluid'), ('O: Other', 'average'), ('O: Other', 'time'), ('O: Other', 'spent'), ('O: Other', 'in'), ('O: Other', 'the'), ('O: Other', 'operating'), ('O: Other', 'room'), ('O: Other




1964
[('P: Condition', 'disseminated'), ('P: Condition', 'small'), ('P: Condition', 'noncleaved-cell'), ('P: Condition', 'lymphoma'), ('P: Condition', 'B-cell'), ('P: Condition', 'leukemia'), ('P: Age', 'Children'), ('P: Age', "'s"), ('P: Condition', 'Cancer'), ('P: Condition', 'disseminated'), ('P: Condition', 'small'), ('P: Condition', 'noncleaved-cell'), ('P: Condition', 'lymphoma'), ('P: Condition', 'B-cell'), ('P: Condition', 'leukemia'), ('I: Other', 'Children'), ('I: Other', "'s"), ('I: Other', 'Cancer'), ('I: Other', 'Group'), ('I: Other', 'trials'), ('P: Sample size', '462'), ('P: Condition', 'disseminated'), ('P: Condition', 'disease'), ('P: Sample size', '49'), ('P: Condition', 'CNS'), ('P: Condition', 'CNS+'), ('P: Condition', 'CNS+'), ('P: Sample size', '28'), ('P: Sample size', 'All'), ('I: Drug', 'protocol-based'), ('I: Drug', 'systemic'), ('I: Drug', 'and'), ('I: Drug', 'intrathecal'), ('I: Drug', 'chemotherapy'), ('I: Physical', 'CNS'), ('I: Physical', 'irradiation'




1801
[('I: Other', 'TOM'), ('I: Other', 'test'), ('P: Condition', 'pervasive'), ('P: Condition', 'developmental'), ('P: Condition', 'disorders'), ('O: Other', 'TOM'), ('O: Other', 'test'), ('P: Condition', 'pervasive'), ('P: Condition', 'developmental'), ('P: Condition', 'disorders'), ('P: Condition', '('), ('P: Condition', 'PDDs'), ('P: Condition', ')'), ('P: Condition', '.'), ('I: Other', 'TOM'), ('I: Other', 'test'), ('P: Sample size', '70'), ('P: Age', 'young'), ('P: Age', 'children'), ('P: Age', 'older'), ('P: Age', 'children'), ('O: Other', 'TOM'), ('O: Other', 'test'), ('P: Age', 'children'), ('P: Sample size', '12'), ('O: Other', 'TOM'), ('O: Other', 'test'), ('P: Age', 'children'), ('P: Condition', 'PDDs'), ('P: Sample size', '10'), ('O: Other', 'interrater'), ('O: Other', 'reliability'), ('O: Other', 'TOM'), ('O: Other', 'test'), ('P: Age', 'children'), ('P: Condition', 'PDDs'), ('P: Sample size', '20'), ('O: Other', 'TOM'), ('O: Other', 'test'), ('O: Other', 'scores'), ('




1124
[('O: Physical', 'Blood'), ('O: Physical', 'pressure'), ('I: Psychological', 'biofeedback'), ('I: Psychological', 'treatment'), ('O: Physical', 'blood'), ('O: Physical', 'pressure'), ('I: Psychological', 'biofeedback'), ('I: Psychological', 'treatment'), ('I: Psychological', '('), ('I: Psychological', 'BF'), ('I: Psychological', ')'), ('P: Sample size', 'Fifteen'), ('P: Condition', 'white-coat'), ('P: Condition', 'hypertensive'), ('P: Sample size', '23'), ('P: Condition', 'essential'), ('P: Condition', 'hypertensive'), ('I: Psychological', 'BF'), ('I: Physical', 'visited'), ('I: Physical', 'the'), ('I: Physical', 'clinic'), ('O: Physical', 'BP'), ('I: Other', 'BF'), ('O: Physical', 'BPs'), ('O: Physical', 'of'), ('O: Physical', 'white-coat'), ('O: Physical', 'hypertensives'), ('O: Physical', 'and'), ('O: Physical', 'essential'), ('O: Physical', 'hypertensives'), ('I: Other', 'BF'), ('O: Physical', 'pulse'), ('O: Physical', 'respiratory'), ('O: Physical', 'rates'), ('O: Physical




1178
[('O: Physical', 'stimulus'), ('O: Physical', 'overselectivity'), ('I: Psychological', 'nonverbal'), ('I: Psychological', 'differential'), ('I: Psychological', 'observing'), ('I: Psychological', 'responses'), ('P: Sample size', 'Three'), ('P: Condition', 'mental'), ('P: Condition', 'retardation'), ('I: Psychological', 'two'), ('I: Psychological', 'sample'), ('I: Psychological', 'stimuli'), ('O: Other', 'accuracy'), ('O: Other', 'scores'), ('O: Other', 'Accuracy'), ('O: Other', 'accuracy'), ('O: Other', 'accuracy'), ('O: Other', 'scores'), ('I: Psychological', 'differential'), ('I: Psychological', 'observing'), ('I: Psychological', 'responses'), ('O: Other', 'accuracy'), ('O: Mental', 'lasting'), ('O: Mental', 'benefits')]


In [12]:
#Resampling four abstracts because it was not actually of that type or non-RCT study.
random.seed(32)
sample = random_sample(1, main_interventions, "I: Surgical")
print(sample)
index_text4 = sample[0][1]
print(index_text4)
print(ann_val[index_text4])
text = texts_val[index_text4][1]
#print(len(text))
doc = nlp1(str(text))
displacy.render(doc, style="ent", jupyter = True, options = optionshier)
random.seed(7)
sample = random_sample(1, main_interventions, "I: Other")
print(sample)
index_text = sample[0][1]
print(index_text)
print(ann_val[index_text])
text = texts_val[index_text][1]
#print(len(text))
doc = nlp1(str(text))
displacy.render(doc, style="ent", jupyter = True, options = optionshier)
random.seed(43)
sample = random_sample(1, main_interventions, "I: Physical")
print(sample)
index_text2 = sample[0][1]
print(index_text2)
print(ann_val[index_text2])
text = texts_val[index_text2][1]
#print(len(text))
doc = nlp1(str(text))
displacy.render(doc, style="ent", jupyter = True, options = optionshier)
random.seed(8)
sample = random_sample(1, main_interventions, "I: Surgical")
print(sample)
index_text3 = sample[0][1]
print(index_text3)
print(ann_val[index_text3])
text = texts_val[index_text3][1]
#print(len(text))
doc = nlp1(str(text))
displacy.render(doc, style="ent", jupyter = True, options = optionshier)
#displacy.serve(doc, style="ent", options = optionshier)

[['I: Surgical', 113]]
113
[('I: Physical', 'stentless'), ('I: Surgical', 'conventional'), ('I: Surgical', 'aortic'), ('I: Surgical', 'valve'), ('I: Surgical', 'replacement'), ('I: Surgical', 'conventional'), ('I: Surgical', 'biological'), ('I: Surgical', 'versus'), ('I: Surgical', 'stentless'), ('I: Surgical', 'aortic'), ('I: Surgical', 'valve'), ('I: Surgical', 'replacement'), ('I: Surgical', '.'), ('I: Physical', 'Stentless'), ('P: Sample size', '106'), ('I: Physical', 'conventional'), ('I: Physical', 'biological'), ('I: Physical', 'aortic'), ('I: Physical', 'valves'), ('P: Sample size', '74'), ('O: Physical', 'aortic'), ('O: Physical', 'valve'), ('O: Physical', 'pathology'), ('O: Physical', ','), ('O: Physical', 'left'), ('O: Physical', 'ventricular'), ('O: Physical', 'function'), ('O: Physical', ','), ('O: Physical', 'and'), ('O: Physical', 'pressure'), ('O: Physical', 'gradients'), ('O: Physical', 'patient'), ('O: Physical', 'annulus'), ('O: Physical', 'index'), ('O: Physical', '

[['I: Other', 183]]
183
[('I: Educational', 'case'), ('I: Educational', 'management'), ('O: Other', 'costs'), ('O: Other', 'of'), ('O: Other', 'health'), ('O: Other', 'care'), ('P: Age', 'older'), ('P: Age', 'population'), ('P: Age', "'s"), ('O: Other', 'costs'), ('O: Other', 'of'), ('O: Other', 'health'), ('O: Other', 'care'), ('P: Age', 'aged'), ('P: Age', '65'), ('P: Age', 'or'), ('P: Age', 'older'), ('P: Sample size', '6409'), ('I: Other', 'Screening'), ('I: Other', 'for'), ('I: Other', 'high'), ('I: Other', 'risk'), ('I: Other', 'and'), ('I: Other', 'provision'), ('I: Other', 'of'), ('I: Educational', 'social'), ('I: Educational', 'work-based'), ('I: Educational', 'case'), ('I: Educational', 'management'), ('I: Other', '.'), ('O: Other', 'Volume'), ('O: Other', 'and'), ('O: Other', 'cost'), ('O: Other', 'of'), ('O: Other', 'hospital'), ('O: Other', ','), ('O: Other', 'physician'), ('O: Other', ','), ('O: Other', 'case'), ('O: Other', 'management'), ('O: Other', ','), ('O: Other', 

[['I: Physical', 8]]
8
[('P: Condition', 'implant-retained'), ('P: Condition', 'mandibular'), ('P: Condition', 'overdentures'), ('P: Condition', '.'), ('I: Educational', 'implant-retained'), ('I: Educational', 'overdentures'), ('O: Physical', 'masticatory'), ('O: Physical', 'performance'), ('O: Physical', ','), ('O: Physical', 'and'), ('O: Physical', 'the'), ('O: Physical', 'subjective'), ('O: Physical', 'chewing'), ('O: Physical', 'experience'), ('O: Physical', '.'), ('I: Physical', 'implant-supported'), ('I: Physical', 'mandibular'), ('I: Physical', 'overdenture'), ('I: Physical', 'transmandibular'), ('I: Physical', 'implant'), ('I: Physical', '('), ('I: Physical', 'TMI'), ('I: Physical', ')'), ('O: Physical', 'Masticatory'), ('O: Physical', 'performance'), ('O: Physical', 'as'), ('O: Physical', 'well'), ('O: Physical', 'as'), ('O: Physical', 'chewing'), ('O: Physical', 'experience'), ('O: Physical', 'masticatory'), ('O: Physical', 'performance'), ('O: Physical', 'and'), ('O: Physica

[['I: Surgical', 201]]
201
[('O: Physical', 'Outcomes'), ('O: Physical', 'of'), ('O: Physical', 'early'), ('O: Physical', 'endovascular'), ('O: Physical', 'versus'), ('O: Physical', 'surgical'), ('O: Physical', 'treatment'), ('O: Physical', 'cerebral'), ('O: Physical', 'aneurysms'), ('O: Physical', '.'), ('I: Surgical', 'surgical'), ('I: Surgical', 'clipping'), ('I: Surgical', 'and'), ('I: Surgical', 'endovascular'), ('I: Surgical', 'treatment'), ('P: Sample size', 'One'), ('P: Sample size', 'hundred'), ('P: Sample size', 'nine'), ('I: Surgical', 'surgical'), ('I: Drug', 'endovascular'), ('I: Physical', 'Follow-up'), ('I: Physical', 'angiography'), ('O: Other', 'recovery'), ('O: Physical', 'severe'), ('O: Physical', 'disability'), ('O: Physical', 'or'), ('O: Physical', 'were'), ('O: Physical', 'in'), ('O: Physical', 'a'), ('O: Physical', 'vegetative'), ('O: Physical', 'state'), ('O: Mortality', 'died'), ('O: Other', 'recovery'), ('O: Mental', 'neuropsychological'), ('O: Mental', 'test'

In [13]:
#Remove incorrectly sampled abstracts
del indices[8]
del indices[7]
del indices[6]
del indices[3]

#New correct sample:
full_index_list = [index_text, index_text2, index_text3, index_text4] + indices #Append
for index in full_index_list:
    print(texts_val[index])

['10968308.txt'
 "The effect of case management on the costs of health care for enrollees in Medicare Plus Choice plans: a randomized trial.\n\nOBJECTIVE To measure the effects of case management on an older population's costs of health care.\nDESIGN A 1-year randomized controlled trial.\nSETTING Multiple sites of care in San Francisco, California.\nPARTICIPANTS Patients aged 65 or older of primary care physicians in a large provider organization bearing financial risk for their care (n = 6409).\nINTERVENTION Screening for high risk and provision of social work-based case management.\nOUTCOME MEASURES Volume and cost of hospital, physician, case management, and other health-related services.\nRESULTS The experimental group used more case management services than the control group (0.09 vs. 0.02 months per person, P<.001). The experimental group's average total payments for health care were slightly lower ($3148 vs $3277, P = .40).\nCONCLUSIONS This study provides no statistically signi

#### Metrics off the sampled abstracts of round 2 evaluated against the ground truth


In [14]:
stand_off_dev = stand_off_train[:val_size]
stand_off_dev = np.array(stand_off_dev)
stand_off_dev = stand_off_dev[full_index_list]
data = []
for i,ann in enumerate(stand_off_dev):
    mydict = {"entities": ann}
    data.append((str(texts_val[full_index_list[i]][1]),mydict))
    
from spacy.training.example import Example

examples = []
for text, ann in data:
    doc = nlp1.make_doc(text)
    examples.append(Example.from_dict(doc, ann))
for i in range(12):
    print(f"Metrics for document with filename {texts_val[i][0]}: {nlp1.evaluate([examples[i]])} \n")



Metrics for document with filename 10036953.txt: {'token_acc': 1.0, 'token_p': 1.0, 'token_r': 1.0, 'token_f': 1.0, 'ents_p': 0.8260869565217391, 'ents_r': 0.3333333333333333, 'ents_f': 0.475, 'ents_per_type': {'P: Age': {'p': 0.8, 'r': 0.5714285714285714, 'f': 0.6666666666666666}, 'P: Sample size': {'p': 1.0, 'r': 1.0, 'f': 1.0}, 'I: Educational': {'p': 0.6666666666666666, 'r': 0.6666666666666666, 'f': 0.6666666666666666}, 'O: Other': {'p': 1.0, 'r': 0.2857142857142857, 'f': 0.4444444444444445}, 'P: Condition': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'I: Other': {'p': 0.0, 'r': 0.0, 'f': 0.0}}, 'speed': 3868.074188683514} 

Metrics for document with filename 10037531.txt: {'token_acc': 1.0, 'token_p': 1.0, 'token_r': 1.0, 'token_f': 1.0, 'ents_p': 0.36363636363636365, 'ents_r': 0.2553191489361702, 'ents_f': 0.29999999999999993, 'ents_per_type': {'P: Condition': {'p': 1.0, 'r': 0.5, 'f': 0.6666666666666666}, 'I: Physical': {'p': 0.3333333333333333, 'r': 0.625, 'f': 0.43478260869565216}, 'I: Su